# Beautiful Lorenz attractors

*NB : This notebook is initially a post from my [personal blog](http://colasdroin.eu/DataScienceBlog/).*

<img src="http://rsta.royalsocietypublishing.org/content/roypta/369/1956/4705/F2.large.jpg" width="400" height="200" />


Wonder how I made the [animation of the Lorenz attractor on the welcome page](http://colasdroin.eu/DataScienceBlog/)? It's actually really simple. I started from the [fantastic tutorial given on Jake VDP blog](https://jakevdp.github.io/blog/2013/02/16/animating-the-lorentz-system-in-3d/) and made a few customizations to obtain a more aesthetic result. Let's first make a few imports to facilitate our life later.

In [1]:
import numpy as np
from scipy import integrate
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation
import seaborn as sns
from IPython.display import HTML

The first step is then to define the function used to simulate the attractor. I here kept pretty much everything as in the tutorial.

In [2]:
def lorentz_deriv( l_coor , t0, sigma=10., beta=8./3, rho=28.0):
    """Compute the time-derivative of a Lorentz system."""
    [x, y, z] = l_coor
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

I then set a few parameters that will enable to tweak the result:

In [3]:
#parameters
N_trajectories = 20
size_queue = 500
T = 8
N = 1000

Here, `size_queue` will be used to set the length of the trajectories before it gets progressively deleted. `T` corresponds to the length of the simulation, and `N` to the number of timesteps we use. Now, we're facing a chaotic system with a [maximum Lyapunov exponent of about 0.9](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.33.398&rep=rep1&type=pdf), which means that the trajectories are extremely sensitive to the initial condition. This is actually the phenomenon I wanted to highlight with my animation, so I'm going to choose initial conditions which are close to each other, such that we can view them diverge. Since, as we'll see later, we will observe the simulation mainly from the x-y plan, I'm only going to slightly perturb the z coordinate. 

In [4]:
#initial condition
x0 = np.array([[0.1, 0.1, 0]]*N_trajectories)
x0[:,2] = np.linspace(0, 30, N_trajectories)

I can then solve for the trajectories.

In [5]:
# Solve for the trajectories
t = np.linspace(0, T, N)
x_t = np.asarray([integrate.odeint(lorentz_deriv, x0i, t)
                  for x0i in x0])

print('Trajectories simulated.')

Trajectories simulated.


I then use the power of seaborn to create a 2-D array of colors, the second dimension being circular and corresponding to the basic [hls palette](https://seaborn.pydata.org/tutorial/color_palettes.html), while the first dimension sets the saturation of the image.

In [6]:
array_colors = [sns.hls_palette(N, l=.6, s=x) for x in np.linspace(0.4,0.9, N_trajectories)]

As one can see, the first dimension will be used to change the color saturation according to the trajectory index, while the second one will evolve with the time iterations. We then just need to prepare the figure for the simulation.

In [7]:
# Set up figure & 3D axis for animation
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1], projection='3d')
ax.axis('off')

# set up lines and points
lines = [ax.plot([], [], [], '-', c=c[0], alpha = 0.7)[0] for c in array_colors]
pts = [ax.plot([], [], [], 'o', c=c[0])[0] for c in array_colors]

# prepare the axes limits
ax.set_xlim((-25, 25))
ax.set_ylim((-35, 35))
ax.set_zlim((-10, 40))

# set point-of-view: specified by (altitude degrees, azimuth degrees)
ax.view_init(30, 0)

Now, one must define how the lines and points we've declared will evolve in time. Not that we change the color of each trajectory at each iteration.

In [8]:
def animate(i):
    t = int(i/N*T)
    for idx, (line, pt, xi) in enumerate(zip(lines, pts, x_t)):
        x, y, z = xi[:i].T

        line.set_data(x[-size_queue:], y[-size_queue:])
        line.set_3d_properties(z[-size_queue:])
        line.set_color(array_colors[idx][i])

        pt.set_data(x[-1:], y[-1:])
        pt.set_3d_properties(z[-1:])
        pt.set_color(array_colors[idx][i])
        
    ax.view_init(30, 0.3 * i)
    fig.canvas.draw()
    return lines + pts

In [9]:
anim = animation.FuncAnimation(fig, animate, frames=N, interval=30, blit=True)
HTML(anim.to_html5_video())

Quite nice, isn't it? However, I wanted to dig a little bit deeper, and to color the tail of a trajectory according to its 'age'. One would normally use [LineCollection](https://matplotlib.org/examples/pylab_examples/multicolored_line.html) to do that ; however, this doesn't work in 3D :'(

The good thing is that there exist a [3D version of LineCollection](https://matplotlib.org/devdocs/api/_as_gen/mpl_toolkits.mplot3d.art3d.Line3DCollection.html). Let's first import the packages needed.

In [10]:
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

Defining the color is now a bit more complicated than before. We're going to color lead points in black, and color the trajectories as progressively shifting from black to yellow depending on the age of the point considered. To do that, we define a norm to map the data points of the trajectories (whose indexes range from `0` to `size_queue`) to the colormap `'hot'` from matplotlib.

In [11]:
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1], projection='3d')
ax.axis('off')

# prepare the axes limits
ax.set_xlim((-25, 25))
ax.set_ylim((-35, 35))
ax.set_zlim((-10, 40))

# set point-of-view: specified by (altitude degrees, azimuth degrees)
ax.view_init(30, 0)

pts = [ax.plot([], [], [], 'o', c=c, alpha = 0.8)[0] for c in ['black']*N_trajectories]

# Create a continuous norm to map from data points to colors
norm = plt.Normalize(0, 1)
lines = [Line3DCollection([], cmap='hot', norm=norm, lw = 2, alpha = 0.6) for n in range(N_trajectories)]
for line in lines:
    line.set_array(np.linspace(1,0.,size_queue))
    ax.add_collection(line)

The animation is now a bit more complicated, since we need to specify the color of every segment (linking to points of the trajectory), for every trajectory. We also need to take into account the fact that the trajectories are initally less than `size_queue` long.

In [12]:
def animate(i):
    for line, pt, xi in zip(lines, pts, x_t):
        x, y, z = xi[:i].T
        #pad with nan to properly handle colorization
        if len(x)<size_queue:
            xt = np.concatenate([(size_queue-len(x))*[np.nan],x])
            yt = np.concatenate([(size_queue-len(x))*[np.nan],y])
            zt = np.concatenate([(size_queue-len(x))*[np.nan],z])
        else:
            xt = x
            yt = y
            zt = z
        points = np.array([xt[-size_queue:], yt[-size_queue:], zt[-size_queue:]]).T.reshape(-1, 1, 3)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        line.set_segments(segments)
        pt.set_data(x[-1:], y[-1:])
        pt.set_3d_properties(z[-1:])
    ax.view_init(30, 0.3 * i)
    fig.canvas.draw()
    return lines + pts

Let see what this looks like!

In [13]:
anim = animation.FuncAnimation(fig, animate, frames=N, interval=30, blit=True)
HTML(anim.to_html5_video())

Note that the simulation is now way longer, but the result is pretty awesome! Let's wrap this thing up. The full code for the "Rainbow attractor" looks like this:

In [14]:
import numpy as np
from scipy import integrate
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation
import seaborn as sns

def lorentz_deriv( l_coor , t0, sigma=10., beta=8./3, rho=28.0):
    """Compute the time-derivative of a Lorentz system."""
    [x, y, z] = l_coor
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

#parameters
N_trajectories = 20
size_queue = 500
T = 80
N = 10000

#initial condition
x0 = np.array([[0.1, 0.1, 0]]*N_trajectories)
x0[:,2] = np.linspace(0, 30, N_trajectories)

# Solve for the trajectories
t = np.linspace(0, T, N)
x_t = np.asarray([integrate.odeint(lorentz_deriv, x0i, t) for x0i in x0])

# Define the 2D array of colors
array_colors = [sns.hls_palette(N, l=.6, s=x) for x in np.linspace(0.4,0.9, N_trajectories)]

# Set up figure & 3D axis for animation
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1], projection='3d')
ax.axis('off')

# set up lines and points
lines = [ax.plot([], [], [], '-', c=c[0], alpha = 0.7)[0] for c in array_colors]
pts = [ax.plot([], [], [], 'o', c=c[0])[0] for c in array_colors]

# prepare the axes limits
ax.set_xlim((-25, 25))
ax.set_ylim((-35, 35))
ax.set_zlim((-10, 40))

# set point-of-view: specified by (altitude degrees, azimuth degrees)
ax.view_init(30, 0)

# animation function.  This will be called sequentially with the frame number
def animate(i):
    t = int(i/N*T)
    for idx, (line, pt, xi) in enumerate(zip(lines, pts, x_t)):
        x, y, z = xi[:i].T

        line.set_data(x[-size_queue:], y[-size_queue:])
        line.set_3d_properties(z[-size_queue:])
        line.set_color(array_colors[idx][i])

        pt.set_data(x[-1:], y[-1:])
        pt.set_3d_properties(z[-1:])
        pt.set_color(array_colors[idx][i])

    ax.view_init(30, 0.3 * i)
    fig.canvas.draw()
    return lines + pts

# instantiate the animator.
anim = animation.FuncAnimation(fig, animate, frames=N, interval=30, blit=True)

#Save as mp4. This requires mplayer or ffmpeg to be installed
#anim.save('lorenz_rainbow.mp4', writer = 'ffmpeg', fps=3, extra_args=['-vcodec','libx264'], dpi = 250)

#Or directly show the result (doesn't work with Jupyter)
#plt.show()

And the full code for the "fire attractor" is the following:

In [15]:
import numpy as np
from scipy import integrate
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation
import seaborn as sns
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

def lorentz_deriv( l_coor , t0, sigma=10., beta=8./3, rho=28.0):
    """Compute the time-derivative of a Lorentz system."""
    [x, y, z] = l_coor
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

#parameters
N_trajectories = 20
size_queue = 500
T = 80
N = 10000

#initial condition
x0 = np.array([[0.1, 0.1, 0]]*N_trajectories)
x0[:,2] = np.linspace(0, 30, N_trajectories)

# Solve for the trajectories
t = np.linspace(0, T, N)
x_t = np.asarray([integrate.odeint(lorentz_deriv, x0i, t) for x0i in x0])

# Set up figure & 3D axis for animation
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1], projection='3d')
ax.axis('off')

# Define the time-dependant colors
colors = ['black']*N_trajectories
pts = [ax.plot([], [], [], 'o', c=c, alpha = 0.8)[0] for c in ['black']*N_trajectories]

# Create a continuous norm to map from data points to colors
norm = plt.Normalize(0, 1)
lines = [Line3DCollection([], cmap='hot', norm=norm, lw = 2, alpha = 0.6) for n in range(N_trajectories)]
for line in lines:
    line.set_array(np.linspace(1,0.,size_queue))
    ax.add_collection(line)

# prepare the axes limits
ax.set_xlim((-25, 25))
ax.set_ylim((-35, 35))
ax.set_zlim((-10, 40))

# set point-of-view: specified by (altitude degrees, azimuth degrees)
ax.view_init(30, 0)

# animation function.  This will be called sequentially with the frame number
def animate(i):
    for line, pt, xi in zip(lines, pts, x_t):
        x, y, z = xi[:i].T
        if len(x)<size_queue:
            xt = np.concatenate([(size_queue-len(x))*[np.nan],x])
            yt = np.concatenate([(size_queue-len(x))*[np.nan],y])
            zt = np.concatenate([(size_queue-len(x))*[np.nan],z])
        else:
            xt = x
            yt = y
            zt = z
        points = np.array([xt[-size_queue:], yt[-size_queue:], zt[-size_queue:]]).T.reshape(-1, 1, 3)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        line.set_segments(segments)

        pt.set_data(x[-1:], y[-1:])
        pt.set_3d_properties(z[-1:])

    ax.view_init(30, 0.3 * i)
    fig.canvas.draw()
    return lines + pts

# instantiate the animator.
anim = animation.FuncAnimation(fig, animate, frames=N, interval=30, blit=True)

#Save as mp4. This requires mplayer or ffmpeg to be installed
#anim.save('lorenz_fire.mp4', writer = 'ffmpeg', fps=3, extra_args=['-vcodec','libx264'], dpi = 250)

#Or directly show the result (doesn't work with Jupyter)
#plt.show()